In [ ]:
import networkx as nx
import numpy as np

file_path = "sacchpomb_multiplex_2.edges"
with open(file_path, 'r') as file:
    lines = file.readlines()
layer_graphs = {}
G = nx.MultiGraph()

for line in lines:
    layer, node1, node2, weight = map(int, line.strip().split())
    G.add_node(node1, layer=layer)
    G.add_node(node2, layer=layer)
    G.add_edge(node1, node2, layer=layer, weight=weight)
    if layer not in layer_graphs:
        layer_graphs[layer] = nx.MultiGraph()
    layer_graphs[layer].add_edge(node1, node2, weight=weight)

num_nodes_per_layer = {}
degree_sum_per_layer = {}

for layer, layer_graph in layer_graphs.items():
    num_nodes_per_layer[layer] = len(layer_graph.nodes())
    degrees = dict(layer_graph.degree())
    degree_sum = sum(degrees.values())
    degree_sum_per_layer[layer] = degree_sum
total_degree_sum = sum(degree_sum_per_layer.values())
degree_sum_normalized = {layer: degree_sum / total_degree_sum for layer, degree_sum in degree_sum_per_layer.items()}
total_normalized_degree_sum = sum(degree_sum_normalized.values())
degree_sum_normalized_normalized = {layer: degree / total_normalized_degree_sum for layer, degree in degree_sum_normalized.items()}

num_nodes_array = list(num_nodes_per_layer.values())
degree_sum_array = list(degree_sum_per_layer.values())
degree_sum_normalized_array = list(degree_sum_normalized.values())
degree_sum_normalized_normalized_array = list(degree_sum_normalized_normalized.values())
b = num_nodes_array
E = degree_sum_array
N = 875
ANCPC = [2 * (E_i + b_i) / (N * (N - 1)) for E_i, b_i in zip(E, b)]
NAE = degree_sum_normalized_normalized_array
result2 = [NAE_i + ANCPC_i for NAE_i, ANCPC_i in zip(NAE, ANCPC)]
file_path = "sacchpomb_multiplex_2.edges"
with open(file_path, 'r') as file:
    lines = file.readlines()

layer_graphs = {}
G = nx.MultiGraph()
for line in lines:
    layer, node1, node2, weight = map(int, line.strip().split())
    G.add_node(node1, layer=layer)
    G.add_node(node2, layer=layer)
    G.add_edge(node1, node2, layer=layer, weight=weight)

    if layer not in layer_graphs:
        layer_graphs[layer] = nx.Graph()  

for edge in G.edges(data=True):
    node1, node2, data = edge
    layer = data['layer']
    weight = data['weight']
    layer_graphs[layer].add_edge(node1, node2, weight=weight)
    
max_node_id = max(node_id for layer_graph in layer_graphs.values() for edge in layer_graph.edges() for node_id in edge)
all_nodes = list(range(max_node_id + 1))

betweenness_centralities = {}
for layer, layer_graph in layer_graphs.items():
    betweenness_centrality = nx.betweenness_centrality(layer_graph)
    betweenness_centralities[layer] = {node: betweenness_centrality.get(node, 0) for node in all_nodes}
layer_weights = result2
num_layers = len(layer_graphs)
node_values = {}

for node in all_nodes:
    value = sum(( betweenness_centralities[layer][node]) * layer_weights[layer-1]
                for layer in range(1, num_layers + 1))
    node_values[node] = value

node_values_array = np.array(list(node_values.values()))
result_string = ','.join(map(str, node_values_array))
print(result_string)

In [ ]:
import numpy as np

def calculate_m(adjacency_matrix):
    A = np.array(adjacency_matrix)
    num_nodes = len(A)
    pci_values = []
    for i in range(num_nodes):
        degree_i = np.sum(A[i, :])
        pci_i = degree_i / (1 + np.sum(A[:, i]))
        pci_values.append(pci_i)
    return pci_values

def find_max_pci_index(pci_values):
    neighbors = [np.nonzero(row)[0] for row in adjacency_matrix]
    max_pci_indices = []
    for i, pci in enumerate(pci_values):
        neighbors_degrees = [np.sum(adjacency_matrix[neighbor, :]) for neighbor in neighbors[i]]
        k = 0
        while k < len(neighbors_degrees) and neighbors_degrees[k] >= k + 1:
            k += 1
        max_pci_indices.append(k)
    return max_pci_indices
file_path = 'xxx'
adjacency_matrix = np.loadtxt(file_path)
pci_values = calculate_pci(adjacency_matrix)
max_pci_indices = find_max_pci_index(pci_values)
result_array = np.array(max_pci_indices)
result_string = ','.join(map(str, result_array))
print(result_string)

In [ ]:
import networkx as nx


file_path = "network_edges.edges"
with open(file_path, 'r') as file:
    lines = file.readlines()
layer_graphs = {}
G = nx.MultiGraph()
for line in lines:
    layer, node1, node2, weight = map(int, line.strip().split())
    G.add_edge(node1, node2, layer=layer, weight=weight)
    if layer not in layer_graphs:
        layer_graphs[layer] = nx.Graph() 
    layer_graphs[layer].add_edge(node1, node2, weight=weight)


all_nodes = set(node for layer_graph in layer_graphs.values() for node in layer_graph.nodes())
max_node_id = max(node for layer_graph in layer_graphs.values() for node in layer_graph.nodes())
all_nodes |= set(range(1, max_node_id + 1))

def calculate_h_index(node, layer_graphs):
    neighbor_degrees = []

    for layer in layer_graphs:
        if node in layer_graphs[layer]:
            neighbors = layer_graphs[layer].neighbors(node)
            neighbor_degrees.extend([layer_graphs[layer].degree(neighbor) for neighbor in neighbors])

    neighbor_degrees.sort(reverse=True)

    h_index = 0
    for i, degree in enumerate(neighbor_degrees):
        if degree >= i + 1:
            h_index = i + 1
        else:
            break
    
    return h_index

h_indices = [calculate_h_index(node, layer_graphs) for node in sorted(all_nodes)]
output = ', '.join(map(str, h_indices))
print(output)

In [ ]:
import numpy as np

def load_adjacency_matrix(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    adjacency_matrix = []
    for line in lines:
        row = list(map(int, line.strip().split()))
        adjacency_matrix.append(row)
    return np.array(adjacency_matrix)

def svt_centrality(A):
    num_nodes = A.shape[0]
    u1 = np.ones(num_nodes)  
    u2 = np.ones(num_nodes)   

    for _ in range(100): 
        new_u1 = A @ u2  
        new_u1 /= np.linalg.norm(new_u1) if np.linalg.norm(new_u1) != 0 else 1  
        new_u2 = A.T @ new_u1  
        new_u2 /= np.linalg.norm(new_u2) if np.linalg.norm(new_u2) != 0 else 1  
        u1, u2 = new_u1, new_u2
    U = u1 * u2
    return u1, u2, U

file_path = 'Homo.txt'  
A = load_adjacency_matrix(file_path)

u1, u2, svt_scores = svt_centrality(A)

print(",".join(f"{score:.9f}" for score in svt_scores))


In [ ]:
import pandas as pd
import networkx as nx
import numpy as np

data = pd.read_csv('London_multiplex.edges', delim_whitespace=True, header=None, names=['layer', 'node1', 'node2', 'weight'])
max_node = max(data['node1'].max(), data['node2'].max())+1
nodes = set(range( max_node))
layers = data.groupby('layer')
graphs = {}

for layer, edges in layers:
    G = nx.Graph()
    G.add_weighted_edges_from(edges[['node1', 'node2', 'weight']].values)
    graphs[layer] = G

all_nodes_in_layers = set(data['node1']).union(set(data['node2']))
isolated_nodes = nodes - all_nodes_in_layers
node_mass = {node: 0 for node in nodes}
overlap_degree = {node: 0 for node in nodes}

for layer, G in graphs.items():
    for node in G.nodes():
        if node in overlap_degree:
            overlap_degree[node] += 1
        else:
            overlap_degree[node] = 1

for node in nodes:
    if node not in overlap_degree:
        overlap_degree[node] = 0

node_mass = {node: 0 for node in nodes}
for layer, G in graphs.items():
    for node in G.nodes():
        degree = G.degree(node)
        if node not in node_mass:
            node_mass[node] = 0
        if node not in overlap_degree:
            overlap_degree[node] = 0
        node_mass[node] += degree - overlap_degree[node]

R = 1
node_distances = {}

for layer, G in graphs.items():
    shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G))
    for i in nodes:
        for j in nodes:
            if i != j:
                distance = shortest_paths.get(i, {}).get(j, R)
                if distance > R:
                    distance = 0
                if (i, j) not in node_distances:
                    node_distances[(i, j)] = 0
                
                node_distances[(i, j)] += distance
final_distances = {k: 1/v if v != 0 else 0 for k, v in node_distances.items()}
distance_list = [f"{i}-{j}:{final_distances[(i, j)]}" for i, j in sorted(final_distances)]
distance_str = ','.join(distance_list)
gravitational_centrality = {}
for node in nodes:
    centrality = 0
    for other_node in nodes:
        if node != other_node:
            mass_product = node_mass[node] * node_mass[other_node]
            distance = final_distances.get((node, other_node), 0)
            if distance > 0:
                centrality += mass_product / distance
    
    gravitational_centrality[node] = centrality
print("WGC: ", gravitational_centrality)